In [1]:
from pgtools import gff_parser
import os
import pandas as pd

In [4]:
cds_cvg = {
"Pneumococcus":85.12,
"Klebsiella":84.94,
"E. coli":84.41,
"E. coli fragmented": 81.40,
"E. coli contaminated": 84.22
}

In [7]:
def datasets_mapping(x):
    if x.startswith("sim"):
        x= x.replace("sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1", "E. coli ")
    else:
        x= x.replace("_subset", "")
        x= x.replace("GPSC", "Pneumococcus")
        x = x.replace("klebsiella", "Klebsiella")
    return x.strip().replace("_","")

In [2]:
summaries_dir="/home/pampuch/studia/magisterka/final_summaries/dataset_summaries/"

In [25]:
columns = ["Dataset","Średnia liczba contigów w genomie", "Std liczby contigów w genomie", "Średnie N50 genomu", "Std N50 genomu"]
rows = []
for f in os.listdir(summaries_dir):
    print(f)
    if f == "summary.csv":
        continue
    row = []
    df = pd.read_csv(os.path.join(summaries_dir, f))
    row.append(datasets_mapping(f.split(".")[0]))
    print(df.columns)
    row += [df["contigs_n"].mean(), df["contigs_n"].std(), df["N50"].mean(), df["N50"].std()]
    rows.append(row)
print(rows)

sim_gr_1e-11_lr_1e-12_mu_1e-14_rep1.csv
Index(['genome', 'genome_size', 'contigs_n', 'N50'], dtype='object')
sim_gr_1e-12_lr_1e-12_mu_1e-14_rep1.csv
Index(['genome', 'genome_size', 'contigs_n', 'N50'], dtype='object')
GPSC_subset.csv
Index(['genome', 'genome_size', 'contigs_n', 'N50'], dtype='object')
klebsiella_subset.csv
Index(['genome', 'genome_size', 'contigs_n', 'N50'], dtype='object')
summary.csv
sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1.csv
Index(['genome', 'genome_size', 'contigs_n', 'N50'], dtype='object')
sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_fragmented.csv
Index(['genome', 'genome_size', 'contigs_n', 'N50'], dtype='object')
sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_contaminated.csv
Index(['genome', 'genome_size', 'contigs_n', 'N50'], dtype='object')
[['simgr1e-11lr1e-12mu1e-14rep1', 107.07, 5.8070924938667225, 140561.0, 5801.96580409771], ['simgr1e-12lr1e-12mu1e-14rep1', 145.89, 4.485430961355989, 169733.79, 318.18060099334286], ['Pneumococcus', 124.6, 151.51529365224593, 115445.85, 46288

In [26]:
df_conts = pd.DataFrame(data=rows, columns=columns)
df_conts = df_conts.sort_values(by="Dataset")
for col in df_conts.columns[1:]:
    df_conts[col] = df_conts[col].round(2)
df_conts

,Dataset,Średnia liczba contigów w genomie,Std liczby contigów w genomie,Średnie N50 genomu,Std N50 genomu
4,E. coli,123.37,7.71,130416.64,1765.79
6,E. coli contaminated,137.43,6.78,110721.06,7555.84
5,E. coli fragmented,322.54,8.42,23072.01,1358.08
3,Klebsiella,780.40,481.44,74041.30,60627.55
2,Pneumococcus,124.60,151.52,115445.85,46288.96
0,simgr1e-11lr1e-12mu1e-14rep1,107.07,5.81,140561.00,5801.97
1,simgr1e-12lr1e-12mu1e-14rep1,145.89,4.49,169733.79,318.18


In [27]:
df_conts.to_csv(os.path.join(summaries_dir, "summary.csv"))

In [5]:
cds_cvg

{'Pneumococcus': 85.12,
 'Klebsiella': 84.94,
 'E. coli': 84.41,
 'E. coli fragmented': 81.4,
 'E. coli contaminated': 84.22}

In [7]:
all_gffs_dir = "/home/pampuch/studia/magisterka/test_data/gffs/"


In [11]:
for dataset in os.listdir(all_gffs_dir):
    print(f'"{dataset}",') 

"sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_fragmented",
"sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_contaminated",
"klebsiella_subset",
"sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1",
"sim_gr_1e-12_lr_1e-12_mu_1e-14_rep1",
"sim_gr_1e-11_lr_1e-12_mu_1e-14_rep1",
"GPSC_subset",


In [14]:
datasets = [
"sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1",
"sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_contaminated",
"sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_fragmented",
"klebsiella_subset",
"GPSC_subset",
]

In [16]:
kleb_gff = os.path.join(all_gffs_dir, datasets[3], "gff")

In [73]:
columns = [
"Dataset",
"Liczba genomów",
"Sumaryczna długość genomów",
"Sumaryczna liczba kontigów",
"Średnia długość kontigu",
"Średni rozmiar genomu",
"Średnia długość kontigu w genomie",
"Średnia liczba kontigów w genomie",
"Pokrycie genomów przez CDS (%)",
]

In [74]:
def basic_stats(genome_gffs):
    all_mean_cont_lens = []
    # all_mean_cont_n = []
    all_cont_lens = []
    all_contig_n = []
    all_genome_lens = []
    genomes_n = 0
    # all_len = 0
    gff_all = gff_parser.parse_GFFs_dir(genome_gffs, gff_simple=False)
    for genome, gff in gff_all.items():
        cont_lens = []
        genomes_n += 1
        cont_n = 0
        for scaff in gff.scaffolds:
            cont_n += 1
            cont_lens.append(scaff.length)
            all_cont_lens.append(scaff.length)
        all_genome_lens.append(sum(cont_lens))
        all_contig_n.append(cont_n)
        all_mean_cont_lens.append(sum(cont_lens) / cont_n)

    res_dict = {
        "Liczba genomów": genomes_n,
        "Sumaryczna długość genomów": sum(all_genome_lens),
        "Sumaryczna liczba kontigów": sum(all_contig_n),
        "Średnia długość kontigu": sum(all_cont_lens) / len(all_cont_lens),
        "Średni rozmiar genomu": sum(all_genome_lens) / genomes_n,
        "Średnia długość kontigu w genomie": sum(all_mean_cont_lens) / genomes_n,
        "Średnia liczba kontigów w genomie": sum(all_contig_n) / genomes_n,
        # "Pokrycie genomów przez CDS (%)",
    }

    return res_dict
            


In [25]:
res_dict = basic_stats(kleb_gff)

In [26]:
res_dict

{'Liczba genomów': 20,
 'Sumaryczna długość genomów': 117375178,
 'Sumaryczna liczba kontigów': 15608,
 'Średni rozmiar genomu': 5868758.9,
 'Średnia długość kontigu w genomie': 11384.184298403612,
 'Średnia liczba kontigów w genomie': 780.4}

In [75]:
summary_dicts = {}
for dataset in datasets:
    dir = os.path.join(all_gffs_dir, dataset, "gff")
    summary_dicts[datasets_mapping(dataset)] = basic_stats(dir)

In [40]:
summary_dicts

{'E. coli': {'Liczba genomów': 100,
  'Sumaryczna długość genomów': 347207379,
  'Sumaryczna liczba kontigów': 12337,
  'Średni rozmiar genomu': 3472073.79,
  'Średnia długość kontigu w genomie': 28250.17757423725,
  'Średnia liczba kontigów w genomie': 123.37},
 'E. coli contaminated': {'Liczba genomów': 100,
  'Sumaryczna długość genomów': 348927115,
  'Sumaryczna liczba kontigów': 13743,
  'Średni rozmiar genomu': 3489271.15,
  'Średnia długość kontigu w genomie': 25449.42886845667,
  'Średnia liczba kontigów w genomie': 137.43},
 'E. coli fragmented': {'Liczba genomów': 100,
  'Sumaryczna długość genomów': 347126358,
  'Sumaryczna liczba kontigów': 32254,
  'Średni rozmiar genomu': 3471263.58,
  'Średnia długość kontigu w genomie': 10769.495195841278,
  'Średnia liczba kontigów w genomie': 322.54},
 'Klebsiella': {'Liczba genomów': 20,
  'Sumaryczna długość genomów': 117375178,
  'Sumaryczna liczba kontigów': 15608,
  'Średni rozmiar genomu': 5868758.9,
  'Średnia długość kontigu w

In [76]:
for k in summary_dicts:
    summary_dicts[k]["Pokrycie genomów przez CDS (%)"] =  cds_cvg[k]

In [43]:
summary_dicts

{'E. coli': {'Liczba genomów': 100,
  'Sumaryczna długość genomów': 347207379,
  'Sumaryczna liczba kontigów': 12337,
  'Średni rozmiar genomu': 3472073.79,
  'Średnia długość kontigu w genomie': 28250.17757423725,
  'Średnia liczba kontigów w genomie': 123.37,
  'Pokrycie genomów przez CDS': 84.41},
 'E. coli contaminated': {'Liczba genomów': 100,
  'Sumaryczna długość genomów': 348927115,
  'Sumaryczna liczba kontigów': 13743,
  'Średni rozmiar genomu': 3489271.15,
  'Średnia długość kontigu w genomie': 25449.42886845667,
  'Średnia liczba kontigów w genomie': 137.43,
  'Pokrycie genomów przez CDS': 84.22},
 'E. coli fragmented': {'Liczba genomów': 100,
  'Sumaryczna długość genomów': 347126358,
  'Sumaryczna liczba kontigów': 32254,
  'Średni rozmiar genomu': 3471263.58,
  'Średnia długość kontigu w genomie': 10769.495195841278,
  'Średnia liczba kontigów w genomie': 322.54,
  'Pokrycie genomów przez CDS': 81.4},
 'Klebsiella': {'Liczba genomów': 20,
  'Sumaryczna długość genomów': 

In [51]:
columns

['Dataset',
 'Liczba genomów',
 'Sumaryczna długość genomów',
 'Sumaryczna liczba kontigów',
 'Średni rozmiar genomu',
 'Średnia długość kontigu w genomie',
 'Średnia liczba kontigów w genomie',
 'Pokrycie genomów przez CDS (%)']

In [77]:
all_row_vals = []
for dataset, vals in summary_dicts.items():
    row_vals = [dataset]
    for k in columns[1:]:
        row_vals.append(vals[k])
    all_row_vals.append(row_vals)

In [60]:
all_row_vals

[['E. coli',
  100,
  347207379,
  12337,
  3472073.79,
  28250.17757423725,
  123.37,
  84.41],
 ['E. coli contaminated',
  100,
  348927115,
  13743,
  3489271.15,
  25449.42886845667,
  137.43,
  84.22],
 ['E. coli fragmented',
  100,
  347126358,
  32254,
  3471263.58,
  10769.495195841278,
  322.54,
  81.4],
 ['Klebsiella',
  20,
  117375178,
  15608,
  5868758.9,
  11384.184298403612,
  780.4,
  84.94],
 ['Pneumococcus',
  20,
  45919085,
  2492,
  2295954.25,
  36545.615865109445,
  124.6,
  85.12]]

In [78]:
df_dataset = pd.DataFrame(columns=columns, data=all_row_vals)
for col in df_dataset.columns:
    if col.startswith("Ś"):
        df_dataset[col] = df_dataset[col].round(2)

In [82]:
for col in df_dataset.columns:
    print(f'"{col}",')

"Dataset",
"Liczba genomów",
"Sumaryczna długość genomów",
"Sumaryczna liczba kontigów",
"Średnia długość kontigu",
"Średni rozmiar genomu",
"Średnia długość kontigu w genomie",
"Średnia liczba kontigów w genomie",
"Pokrycie genomów przez CDS (%)",


In [84]:
df_dataset = df_dataset[["Dataset",
"Liczba genomów",
"Sumaryczna długość genomów",
"Średni rozmiar genomu",
"Pokrycie genomów przez CDS (%)",
"Sumaryczna liczba kontigów",
"Średnia liczba kontigów w genomie",
"Średnia długość kontigu",
"Średnia długość kontigu w genomie",
]]

In [93]:
df_dataset

,Dataset,Liczba genomów,Sumaryczna długość genomów,Średni rozmiar genomu,Pokrycie genomów przez CDS (%),Sumaryczna liczba kontigów,Średnia liczba kontigów w genomie,Średnia długość kontigu,Średnia długość kontigu w genomie
0,E. coli,100,347207379,3472073.79,84.41,12337,123.37,28143.58,28250.18
1,E. coli contaminated,100,348927115,3489271.15,84.22,13743,137.43,25389.44,25449.43
2,E. coli fragmented,100,347126358,3471263.58,81.40,32254,322.54,10762.27,10769.50
3,Klebsiella,20,117375178,5868758.90,84.94,15608,780.40,7520.19,11384.18
4,Pneumococcus,20,45919085,2295954.25,85.12,2492,124.60,18426.60,36545.62


In [86]:
df_dataset.to_csv("/home/pampuch/studia/magisterka/final_summaries/basic_dataset_stats.csv")

In [66]:
df_dataset["Średnia długość kontigu w genomie"].round(2)

0    28250.18
1    25449.43
2    10769.50
3    11384.18
4    36545.62
Name: Średnia długość kontigu w genomie, dtype: float64

In [111]:
df_stats = pd.read_csv("/home/pampuch/studia/magisterka/final_summaries/stats.csv")

In [112]:
df_stats

,Dataset,Model,Udział przerw (%),Liczba bloków,Łączna długość zawartych sekwencji,Średni rozmiar bloku,Średnia długość sekwencji w bloku
0,E. coli,Panaroo podstawowy,0.11,4234,292139716,72.90,945.35
1,E. coli,Uzupełnienie Panaroo,62.29,2246,83733481,66.17,213.45
2,E. coli,Panaroo z uzupełnieniem,13.96,6480,375873197,70.57,707.48
3,E. coli,Cactus,1.48,1333,332387749,92.35,2629.29
4,E. coli contaminated,Panaroo podstawowy,0.25,4247,292302476,72.62,945.38
5,E. coli contaminated,Uzupełnienie Panaroo,62.33,2251,82821561,65.82,211.56
6,E. coli contaminated,Panaroo z uzupełnieniem,13.96,6498,375124037,70.27,707.26
7,E. coli contaminated,Cactus,1.33,1550,328136320,90.22,2287.57
8,E. coli fragmented,Panaroo podstawowy,4.72,4256,294386131,72.12,913.90
9,E. coli fragmented,Uzupełnienie Panaroo,65.27,2337,84756187,59.98,211.02


In [96]:
df_stats = df_stats.join(df_dataset.set_index("Dataset")["Sumaryczna długość genomów"], on="Dataset")

In [105]:
df_stats["Pokrycie pangenomu"] = df_stats["Łączna długość zawartych sekwencji"] * (1 - df_stats["Udział przerw (%)"]/100)

In [109]:
df_stats["Pokrycie pangenomu (%)"] = df_stats["Pokrycie pangenomu"] * 100 /df_stats["Sumaryczna długość genomów"]
df_stats["Pokrycie pangenomu (%)"] = df_stats["Pokrycie pangenomu (%)"].round(2)


In [110]:
df_stats

,Dataset,Model,Udział przerw (%),Liczba bloków,Łączna długość zawartych sekwencji,Średni rozmiar bloku,Średnia długość sekwencji w bloku,Sumaryczna długość genomów,Pokrycie pangenomu,Pokrycie pangenomu (%)
0,E. coli,Panaroo podstawowy,0.11,4234,292139716,72.90,945.35,347207379,2.918184e+08,84.05
1,E. coli,Uzupełnienie Panaroo,62.29,2246,83733481,66.17,213.45,347207379,3.157590e+07,9.09
2,E. coli,Panaroo z uzupełnieniem,13.96,6480,375873197,70.57,707.48,347207379,3.234013e+08,93.14
3,E. coli,Cactus,1.48,1333,332387749,92.35,2629.29,347207379,3.274684e+08,94.31
4,E. coli contaminated,Panaroo podstawowy,0.25,4247,292302476,72.62,945.38,348927115,2.915717e+08,83.56
5,E. coli contaminated,Uzupełnienie Panaroo,62.33,2251,82821561,65.82,211.56,348927115,3.119888e+07,8.94
6,E. coli contaminated,Panaroo z uzupełnieniem,13.96,6498,375124037,70.27,707.26,348927115,3.227567e+08,92.50
7,E. coli contaminated,Cactus,1.33,1550,328136320,90.22,2287.57,348927115,3.237721e+08,92.79
8,E. coli fragmented,Panaroo podstawowy,4.72,4256,294386131,72.12,913.90,347126358,2.804911e+08,80.80
9,E. coli fragmented,Uzupełnienie Panaroo,65.27,2337,84756187,59.98,211.02,347126358,2.943582e+07,8.48
